<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Note

강화학습에서는 생소한 개념이 많이 등장한다. 강화학습에서 많이 사용되는 수식들이 내포한 의미를 깊이 이해하기위해 직관적으로 이해가능한 수식으로부터 다양한 알고리즘의 업데이트 공식을 유도해본다.

1.일단 직관적으로 이해가능한 MDP의 3개의 수식으로부터 시작한다.

i) Return G는 현 상태에서 에피소드가 끝날때까지 진행됐을때 얻는 reward의 합으로 discounter factor = $ \gamma $을 적용해 미래보상이 할인돼서 계산된다.

$G_{t}  \triangleq R_{t}+\gamma R_{t+1}+\gamma^{2} R_{t+2}+\gamma^{3} R_{t+3}+ ...$

ii) state value function $ V(S_{t}) $는 상태가치를 나타내는 함수로 현 상태로부터 에피소드가 끝날때까지 진행됬을때 얻을 수 있는 Returnd의 기댓값이다.

$ V(s_{t})  \triangleq \int_{a_{t}:a_{\infty}}G_{t}p(a_t,s_{t+1},a_{t+1}, ...|s_{t})d_{a_{t}:a_{\infty}} $

iii) action value function $Q(s_{t},a_{t})$는 행동가치를 나타내는 함수로 현상태에서 가능한 어떤 행동에서부터 에피소드가 끝날때까지 진행됬을때 얻을 수 있는 Return의 기댓값이다.

$ Q(s_t,a_t) \triangleq \int_{s_{t+1}:a_{\infty}}G_{t}p(s_{t+1},a_{t+1}, ...|s_{t})d_{s_{t+1}:a_{\infty}} $

상태가치함수와 행동가치함수 모두 다차원 확률변수에 대한 조건부기댓값으로 표현되므로 수식으로부터 정의된 개념을 직관적으로 이해할 수 있다.

2.위의 수식을 변형해 Bellman equation을 유도하는 과정에서 새로운 인사이트를 얻는다.

i) 상태가치함수 $V(s_t)$를 행동가치함수$Q(s_{t},a_{t})$로 표현하기.

$p(a_t,s_{t+1},a_{t+1}, ...|s_{t}) = p(s_{t+1},a_{t+1}, ...|s_{t},a_t)p(a_t|s_t)
$로부터 $V(s_t)$를 변형된 식으로 표현하면,

$ V(s_{t})  \triangleq \int_{a_t} \int_{s_{t+1}:a_{\infty}}G_{t}p(s_{t+1},a_{t+1}, ...|s_{t},a_t)d_{s_{t+1}:a_{\infty}} p(a_t|s_t)da_t $로 표혀되고 이식을 다시 행동가치함수$Q(s_{t},a_{t})$로 표현하면

$ \int_{a_t}Q(s_t,a_t)p(a_t|s_t)da_t $로 표현된다.

위 식으로부터 얻을 수 있는 인사이트는 상태가치함수 $V(s_t)$가 행동가치함수$Q(s_{t},a_{t})$의 기댓값으로 표현된다는 점 이다. 그러니까 상태에대한 가치는 그 상태에서 policy에 따라 취할 수 있는 행동가치의 평균, 즉 기댓값인 것 이다.

optimal policy($p^*(a_t,s_t) = \delta(a_t-a_t^*)$)
여기서 $a_t^* = argmax_{a_t}Q^*(s_t,a_t)$

또, 상태가치함수 $V(s_t)$를 maximzie하는 policy(행동정책) ($p(a_t|s_t)$) 즉 optimal policy는 행동가치함수 $Q(s_{t},a_{t})$가 가장 큰값만을 갖게하는 정책(greedy action)임을 알 수 있다.(단, 행동가치함수$Q(s_{t},a_{t})$가 $Q^*(s_{t},a_{t})$인 경우만, 즉 미래시점 행동이 모두 optimal policy에 의해 선택된경우)

ii) 상태가치함수 $V(s_t)$를 다음 시점의 상태가치함수 $V(s_t+1)$로 표현하기.

$ p(a_t,s_{t+1},a_{t+1}, ...|s_{t}) = p(a_{t+1}, ...|s_t,a_t, s_{t+1})p(a_t,s_{t+1}|s_t) $ 에서 다시 MDP의 특성으로 부터

$ p(a_{t+1}, ...|s_t,a_t, s_{t+1})p(a_t,s_{t+1}|s_t) = p(a_{t+1}, ...|s_{t+1})p(a_t,s_{t+1}|s_t) $으로 표현되고 또,

$G_{t}  \triangleq R_{t}+\gamma R_{t+1}+\gamma^{2} R_{t+2}+\gamma^{3} R_{t+3}+ ... = R_t+\gamma G_{t+1} $

위의 두 변현식으로 상태가치함수 $V(s_t)$를 다시 표현하면

$ \int_{a_t,s_{t+1}} \int_{a_{t+1}:a_{\infty}} (R_t+\gamma G_{t+1})p(a_{t+1}, ...|s_{t+1})d_{a_{t+1}:a_{\infty}} p(a_t,s_{t+1}|s_t)d_{a_t,s_{t+1}} $로 표현되고 이를 다시 다음시점의 상태가치함수 $V(s_t+1)$로 표현하면

$ \int_{a_t,s_{t+1}} (R_t+\gamma V(s_{t+1}))p(a_t,s_{t+1}|s_t)d_{a_t,s_{t+1}} $로 표현될 수 있다. 여기서 다시

$ p(a_t,s_{t+1}|s_t) = p(s_{t+1}|s_t,a_t)p(a_t|s_t) $로 변형해 표혀하면

$ \int_{a_t,s_{t+1}} (R_t+\gamma V(s_{t+1}))p(s_{t+1}|s_t,a_t)p(a_t|s_t)d_{a_t,s_{t+1}} $로 표현된다.

위의 수식이 Bellman equation으로 이로부터 얻을 수 있는 인사이트는 상태가치함수 $V(s_t)$가 다음시점의 상태가치함수 $V(s_{t+1})$, transition probablity($p(s_{t+1}|s_t,a_t)$), policy($p(a_t|s_t)$)로 표현된다는 점 이다. 특히 policy는 행동정책으로 상태가치함수를 maximize하는 policy, 즉 optimal policy를 찾는것은 강화학습의 주 목적이다.

비슷한 과정으로 행동가치함수$Q(s_{t},a_{t})$또한 다음 상태의 상태가치함수로 표현되거나 다음상태의 행동가치함수로 표현될 수 있다.

$ Q(a_t,s_t) = \int_{s_{t+1},a_{t+1}}(R_t + \gamma Q(s_{t+1},a_{t+1}))p(s_{t+1}|s_t,a_t)p(a_{t+1}|s_{t+1})d_{s_{t+1},a_{t+1}} $

이 또한 Bellman equation으로 상태가치함수의 Bellman equation과의 차이점은 다음상태의 행동정책의 영향을 받는다는 것 이다.

3.$Q^*$는 어떻게 구해지는가?

i) Monte_Carlo

먼저 큰 수의 법칙

$ E[X]=\int_x xp(x)dx \approx \frac{1}{N}\sum^N_{i=1} x_i $ 에 따라

$ Q(s_t,a_t) \triangleq \int_{s_{t+1}:a_{\infty}}G_{t}p(s_{t+1}:a_{\infty}|s_{t})d_{s_{t+1}:a_{\infty}} \approx \frac{1}{N}\sum G^i_t  $ 으로 근사하 수 있다.

위 수식이 이것이 Monte_carlo method이며 이로부터 얻을 수 있느 인사이트는 어떤상태에서 에피소드를 충분히 많이 진행하면서 얻어진 Q값의 평균으로 Q값을 $ Q^* $로 근사할 수 있다는 것 이다. 한계로는 수식을 보면 알 수 있듯이 에피소드를 끝까지 진행해야만 Q값이 업데이트 된다는 점 이다.

II) Temporal difference

먼저 위에서 유도한 행동가치함수의 Bellman equation

$ Q(a_t,s_t) = \int_{s_{t+1},a_{t+1}}(R_t + \gamma Q(s_{t+1},a_{t+1}))p(s_{t+1}|s_t,a_t)p(a_{t+1}|s_{t+1})d_{s_{t+1},a_{t+1}} $ 에 큰수의 법칙을 적용해

$ \approx \frac{1}{N}\sum^N_{i=1}(R^{(I)}_t + \gamma Q(s^{(i)}_{t+1},a^{(i)}_{t+1})) \triangleq \bar{Q_N} $로 근사할 수 있다.
여기서 R이 sample인 이유는 transition probablity때문으로 어떤 행동에 대한 결과 또한 확률적이기 때문이다.

Temporal difference가 Monte_carlo와 다른점은 Monte_carlo는 에피소드가 끝날때까지 진행해야만 Q값을 엄데이트할 수 있는 반면 Temperal difference는 위 수식에서 보는바와 같이 바로 다음시점의 Q값만으로 현시점의 Q값을 바로 업데이트 가능하다는 점 이다.

! $R^{N}_t + \gamma Q(s^{N}_{t+1}, a^{N}_{t+1}) - \bar{Q}_{N-1}$는 따로 TD error라고 부르고 TD error에서 다음시점의 보상과 Q의 샘플 $ R^{N}_t + \gamma Q(s^{N}_{t+1}, a^{N}_{t+1}) $을 TD-taget이라고 한다.

위 식을 변형해 다시 적으면

$ = \frac{1}{N}(\bar{Q}_{N-1}(N-1) + R^{N}_t + \gamma Q(s^{N}_{t+1}, a^{N}_{t+1})) $로 나타낼수 있고 이를 전개하면

$ = \bar{Q}_{N-1} + \frac{1}{N}(R^{N}_t + \gamma Q(s^{N}_{t+1}, a^{N}_{t+1}) - \bar{Q}_{N-1}) $ 로 표현할 수 있다.

이를 1-Step TD 또는 Incremental monte_carlo update라고 한며 이로부터 얻을 수 있는 인사이트는 큰수의 법칙으로부터 $Q^*$를 근사할때 이전 샘플들의 평균에서 다음 샘플을 합한 평균을 구할때 처음부터 계산하지 않고 약간의 변형으로 이전 평균값과 다음 샘플값만으로 다음평균값을 구할 수 있다는 것 이다.

또, 위식을 다시 변형해

$ \frac{1}{N} = \alpha $ 로 부터

$ (1-\alpha)\bar{Q}_{N-1} + \alpha(R^{N}_t + \gamma Q(s^{N}_{t+1}, a^{N}_{t+1})) $로 표현가능하며 이는 SARSA의 업데이트 공식이다.

MC와  SARSA를 비교해보면 SARSA는 업데이트 공식의 원본 $ Q(a_t,s_t) = \int_{s_{t+1},a_{t+1}}(R_t + \gamma Q(s_{t+1},a_{t+1}))p(s_{t+1}|s_t,a_t)p(a_{t+1}|s_{t+1})d_{s_{t+1},a_{t+1}} $으로부터 알 수 있듯이 다음상태의 Q값이 $ Q^*$가 아닌 이상 업데이트는 완전하지 않으며 이로부터 편향이 발생할 수 있다는 것 이다.(biased sample) 대신 에피소드가 끝날때가지 진행해야하는(unbiased sample) MC는 variance가 너무 커서 수렴이 쉽지 않다는 단점이 있다.

4.Q러닝

Q러닝의 업데이트 공식은 SARSA와 같다. 차이점은 Behaier policy와 Target policy가 다르다는 점이다(off policy). 즉 SARSA는 행동정책과 업데이트 샘플을 선택하는 타겟정책이 같고(on policy) Q러닝은 이것이 다르다는 것 이다.

$ Q(a_t,s_t) = \int_{s_{t+1},a_{t+1}}(R_t + \gamma Q(s_{t+1},a_{t+1}))p(s_{t+1}|s_t,a_t)p(a_{t+1}|s_{t+1})d_{s_{t+1},a_{t+1}} $

$ \approx (1-\alpha)\bar{Q}_{N-1} + \alpha(R^{N}_t + \gamma Q(s^{N}_{t+1}, a^{N}_{t+1})) $

위 수식에서 Target policy는 $ p(a_{t+1}|s_{t+1}) $ Behavior policy는 $ p(s_{t+1}|s_t,a_t) $이다.

SARSA는 현상태에서 행동후 다음 상태에 또한 행동정책에 의해 샘플링된 Q값으로 현 Q값이 업데이트 돼고 Q러닝은 현상태서 행동후 다음 상태에서 현동정책과는 별개로 다른 전략으로 Q값을 선택해 업데이트할 수 있다. 이로부터 얻을 수 있는이점은 행동 정책의 탐험률은 높여서 optimal에 도달할 여러 route를 탐색하면서도 업데이트는 가장 높은 Q값으로 업데이트(greedy polcy)하는것이 가능하다는 것 이다.

정리해 보자면 MDP위에서 이뤄지는 task를 해결하기위해 상태가치함수나 행동가치함수를 최대화하는 전략을 사용할 수 있고 그 방법으로 Monte-carlo method와 Temporal difference이 있다. 그 중 Temporal difference는 on-policy와 off-policy로 나눌 수 있으며 각각 SARSA, Q-learning이라고 한다.

# 새 섹션ㅇㄹ

논문에서 제시하는 기존 접근방법들의 문제점 딥러닝의 도입, 딥러닝 도입의 어려움.

1.고차원 입력 즉 큰 상태공간에서 에이전트를 훈련시키는 것은 강화학습의 오랜 도전과제 주 하나이며 이전의 접근방식들은 인간이 직접 유도한 특성을 사용함으로써 이 특성들의 질에 심하게 의존하는 경향이 있다.

하지만 딥러닝이 발전하며 이해하기힘든 고차원 데이터에서 효율적으로 특서을 추출할 수 있게 되었고 이를 응용해 위에서 제기한 도전과제를 해결하고자 한다.

하지만 딥러닝은 많은양의 라벨링된 데이터가 필요한 반면 강화학습은 라벨링된 데이터 없이 희박하거나 noisy하고 지연된 보상으로부터 학습해야하고 데이터가 독립적으로 가정하는 딥러닝과 달리 강화학습은 상관관계가 큰 sequence를 사용하고 강화학습의 데이터분포는 고정된 딥러닝의 경우와 다르게 행동정책이 업데이트됨에따라 달라지기 때문에 딥러닝 기법을 강화학습에 적용하는데 어려움이 있다.

변형된 Q러닝 알고리즘을 사용한 DQN은 이러한 도전과제를 성공적으로 극복했다.

확률적 경사하강법을 사용해 weight를 업데이트함으로써 데이터의 상관성과 비정상적인 데이터 분포를 극복했다. 또 경험 재생 매커니즘을 사용함을써 훈련 분포를 부드럽게했다.

MDP를 적용하기위한 기반

DQN은 에이전트에게 아타리게임을 훈련시키며 입력으로는 오직 게임화면만 제공받는다. 하지만 한시점의 화면만으로는 현재상황에대해 완전히 이해하는것이 불가능하므로(오브젝트가 어디로 이동하고 있는지 등) 하나의 상태를 여러개의 액션과 상태의 묶음으로 정의한다. 이로써 상태에대한 완전한 관찰과 기본적인 마르코프 성질을 충족시킨다.

DQN에서 에이전트의 학습목적은 앞으로의 보상을 maximize하게 만드는 action을 선택하는 것 이며 이는 곧 optimal q 그러니까 $Q^*(s,a) = max_\pi E[R_t|s_t = s, a_t = a, \pi]$를 찾는것을 목표로 한다는 것 이다.

이는 Bellman equation으로 표현 가능하며 미래의 선택이 optimal한 정책에 의해 이루어 졌다면 현상태의 Q값을 maiximize하는 정책으로 다음시점의 Q값중 가장 큰값을 선택하는 전략을 세울 수 있다.

$ Q^*(s,a) = E_{s' - \varepsilon}[r + \gamma max_{a'}Q^*(s',a')|s,a] $

많은 강화학습 알고리즘에서 이를 반복적으로 업데이트함으로써 Q를 추정한다. 하지만 이는 전혀 실용적이지 않은데 그 이유는 Q값을 각 상태,행동 쌍에대해 따로따로 추정하고 일반화가 전혀이루어지지 않기때문에 상태,행동 쌍이 거의 무한한 환경에서는 사용이 불가능 하기 때문이다.

DQN에서는 이 문제를 해결하기위해 action value function을 근사하는 $Q(s,a;\theta) \approx Q^*(s,a)$를 사용하며 이를 Q-network라고 한다.

Q네트워크를 훈련하기위해 사용하는 Loss function은 $L_i(\theta_i) = E_{s, a-p(.)}[(y_i - Q(s,a;\theta_i))^2]$ 이고 여기서 $ y_i = E_{s'-\varepsilon}[r + \gamma max_{a'}Q(s', a';\theta_{i-1})] $이다 여기서 중요한 지점은 $ \theta_{i-1} $이다. Q-network가 업데이되는동안 TD 타겟은 업데이트돼지않고 고정된 파라미터로 샘플링된다. 이는 학습에 사용될 데이터의 분포가 파라미터에 의존하기 때문이다.

DQN 아키텍쳐의 특징

epcperience replay를 활용한다. 이는 에이전트의 경험 $e_t = (s_t,a_t,r_t,s_{t+1})$을 replay memory D에 저장하며 큐네트워크를 업데이트할때 D에서 과거경험들을 랜덤하게 샘플링해 사용한다.

이 접근방식의 장점은 첫째로 각 스텝의 경험이 잠재적으로 많은 웨이트 업데이트에 사용됨으로써 데이터 효율성을 그대화한다

두번쨰로는 연이은 샘들들을 학습시키는것은 두 데이터 샘플의 강한 상관관계때문에 비효율적일 수 있다. 하지만 이를 랜덤 샘플링으로 극복한다.

세번째로 온폴리시 학습의 단점을 극복할 수 있다. 온폴리시 학습시 행동정책과 타겟정책이 같기때문에 학습이 편향될 수 있지만 경험재생을 통해 모델을 업데이트할때 사용되는 Loss function을 살펴보면 on polcy 학습방법을 사용함을 알 수있고 행동 분포는 미니배치를 사용함을써 평균화되어, 학습을 안정화시키고 파라미터의 진동이나 발산을 방지하 수 있다.

이 방법의 문제점도 있는데 과거경험을 무작위샘플링하는 방법과 메모리가 유한하므로 오래된 경험부터 지워나가는 방식은 모두 각 경험의 중요도 모두 같은 지점에 놓는다는 점이다.

# 코드

In [ ]:
!pip install gymnasium
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]
!pip install imageio
!pip install imageio-ffmpeg

In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt
import imageio
from gymnasium.wrappers import FrameStack, GrayScaleObservation
import torch
from torch import nn, optim
import numpy as np
import torch.optim as optim
import random
from collections import deque
from torch.utils.data import Dataset, DataLoader

In [ ]:
# "DonkeyKong" 게임 환경 생성
env = gym.make('ALE/DonkeyKong-v5', render_mode='rgb_array')

# 정의된 환경에 Frame stack, gray scale warraps를 적용. 순서가 바뀌면 오류가 생김.
env = GrayScaleObservation(env)
env = FrameStack(env, 4)

In [ ]:
env.observation_space

In [ ]:
env.action_space

In [ ]:
obs, info = env.reset()

In [ ]:
print(obs.shape)
print(info)
obs[0]

In [ ]:
# 경험 리플레이 메모리
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append([state, action, reward, next_state, done])

    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.array(state), action, reward, np.array(next_state), done

    def __len__(self):
        return len(self.buffer)

In [ ]:
def epsilon_greedy(state, model, epsilon):
    if random.random() > epsilon:
        with torch.no_grad():
            q_values = model(state)
            action = q_values.max(1)[1].item()
    else:
        action = random.randrange(env.action_space.n)
    return action

# DQN model architecture 정의

1. 논문과 다른점.

i) 논문에 명시된 DQN아키텍쳐는 게임이미지를 그레이스케일로 변환하 스택하는 것 말고도 84*84로 크롭핑하는 전처리과정을 포함하고 있지만 여기서는 그냥 Gymnasium 라이브러리에서 반화하는 게임화면을 그레이스케이로 변환해 스택하는 과정만 포함해 원본 이미지 해상도를 그대로 사용한다.

ii) Bathch normalization 레이어 사용.

게임화면을 원본그대로 사용함에 따라 커볼루션 레이이어 1층, dense레이어 1층이 추가 됨으로 모델이 깊어짐에 따라 컨볼루션레이어에 사이에 bach normalization 레이어를 추가했다.

 *차이점 관찰 필요.

2. Note

DQN 아키텍쳐에서 maxpooling 레이어를 사용하지 않는 이유.

게임환경에서는 작은 이미자 하나가 중요한 정보를 포함할 확률이 있으므로 정보손실을 야기하는 maxpooling 레이어를 DQN아키텍쳐에 사용하는것은 안 좋은 영향이 있을 수 있다.

In [ ]:
class DQN(nn.Module):
    def __init__(self):

        super().__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(4, 16, kernel_size = 8, stride = 4),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU())

        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size = 4, stride = 2),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU())

        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, kernel_size = 4, stride = 2),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU())

        self.fc = nn.Sequential(nn.Linear(5632, 256),
                                nn.ReLU(),
                                nn.Linear(256, 18))



    def forward(self, x):

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = torch.flatten(x,start_dim=1)
        x = self.fc(x)

        return x

In [ ]:
model = DQN()

In [ ]:
# "DonkeyKong" 게임 환경 생성
env = gym.make('ALE/DonkeyKong-v5', render_mode='rgb_array')

# 정의된 환경에 Frame stack, gray scale warraps를 적용. 순서가 바뀌면 오류가 생김.
env = GrayScaleObservation(env)
env = FrameStack(env, 4, )

In [ ]:
obs, info = env.reset()

In [ ]:
obs.shape

In [ ]:
def totensor(obs):

    numpy_array = np.array(obs)  # LazyFrames 객체를 np.array()를 사용하여 NumPy 배열로 변환
    tensor = torch.from_numpy(numpy_array) # NumPy 배열을 PyTorch 텐서로 변환
    tensor = tensor.unsqueeze(0)
    tensor = tensor.float()

    return tensor

In [ ]:
obs = totensor(obs)

In [ ]:
obs.shape

In [ ]:
qs = model(obs)

In [ ]:
def epsilon_greedy(obs, model, epsilon):
    if random.random() > epsilon:
        with torch.no_grad():
            obs = np.array(obs)  # LazyFrames 객체를 np.array()를 사용하여 NumPy 배열로 변환
            obs = torch.from_numpy(obs) # NumPy 배열을 PyTorch 텐서로 변환
            obs = obs.unsqueeze(0)
            obs = obs.float()
            q_values = model(obs)
            action = q_values.max(1)[1].item()
    else:
        action = random.randrange(env.action_space.n)
    return action

In [ ]:
action = epsilon_greedy(obs, model, 0.7)

In [ ]:
action

In [ ]:
nobs, reward, terminated, truncated, info = env.step(action)

In [ ]:
# 경험 리플레이 메모리
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state):
        self.buffer.append([state, action, reward, next_state])

    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.array(state), action, reward, np.array(next_state), done

    def __len__(self):
        return len(self.buffer)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# # 언더샘플링

# # 인스턴스 생성.
# short_term_dataset = CustomDataset(short_term_data, short_term_label)

In [ ]:
# "DonkeyKong" 게임 환경 생성
env = gym.make('ALE/DonkeyKong-v5', render_mode='rgb_array')

# 정의된 환경에 Frame stack, gray scale warraps를 적용. 순서가 바뀌면 오류가 생김.
env = GrayScaleObservation(env)
env = FrameStack(env, 4)

model = DQN()

# 리플레이버퍼 생성
replaybuffer = ReplayBuffer(10000)

for episode in range(1000):

    obs, info = env.reset()
    terminated = False
    truncated = False

    while(terminated == False and truncated == False):

        action = epsilon_greedy(obs, model, 0.5)
        nobs, reward, terminated, truncated, info = env.step(action)
        replaybuffer.push(totensor(obs), action, reward, totensor(nobs))
        obs = nobs

    if len(replaybuffer) > 2000:

        replay = list(replaybuffer.buffer)
        dataset = CustomDataset(replay)
        dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    break


In [ ]:
len(replaybuffer)

In [ ]:
replay = list(replaybuffer.buffer)

In [ ]:
first_tensors = [item[0] for item in replay]

In [ ]:
dataset = CustomDataset(replay)

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
for data in dataloader:
    print(data[3].shape)
    break

In [ ]:
# 강화학습 에이전트
class DQNAgent:
    def __init__(self, input_shape, n_actions, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, lr=1e-4, gamma=0.99, buffer_size=10000, batch_size=32):
        self.model = DQN(input_shape, n_actions)
        self.epsilon = epsilon  # 탐색 비율
        self.epsilon_decay = epsilon_decay  # 탐색 감소율
        self.epsilon_min = epsilon_min  # 최소 탐색 비율
        self.gamma = gamma  # 할인율
        self.replay_buffer = ReplayBuffer(buffer_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.batch_size = batch_size
        self.loss_fn = nn.MSELoss()

    def act(self, state):
        if random.random() < self.epsilon:
            return random.randrange(n_actions)
        else:
            state = torch.FloatTensor(state).unsqueeze(0)
            q_values = self.model(state)
            return np.argmax(q_values.detach().numpy())

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)

        states = torch.FloatTensor(states)
        next_states = torch.FloatTensor(next_states)
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        dones = torch.FloatTensor(dones)

        q_values = self.model(states)
        next_q_values = self.model(next_states)
        q_value = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
        next_q_value = next_q_values.max(1)[0]
        expected_q_value = rewards + self.gamma * next_q_value * (1 - dones)

        loss = self.loss_fn(q_value, expected_q_value.detach())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# 메인 학습 루프
if __name__ == "__main__":
    env = gym.make("Kong-v0")  # 킹콩 환경 초기화


In [ ]:
dqn = DQN()
optimizer = optim.Adam(dqn.parameters(), lr=1e-4)

# Hyperparameters
episodes = 1000
epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 300

epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * np.exp(-1. * frame_idx / epsilon_decay)

# Training loop
for episode in range(episodes):
    obs, info = env.reset()
    total_reward = 0
    terminated, truncated = False
    while not (terminated or truncated): # terminated는 에이전트가 성공이나 실패에 이르렀을경우 참을 반환, truncated는 시간제한 초과시 참을 반환
        epsilon = epsilon_by_frame(episode)
        action = epsilon_greedy(state, q_network, epsilon)
        obs, reward, terminated, truncated, info = env.step(0)
        total_reward += reward
    print(f"Episode: {episode}, Total Reward: {total_reward}")

In [ ]:
img = env.render()
plt.imshow(img)
plt.axis('off')

In [ ]:
frames = []  # 이미지를 저장할 리스트
env = gym.make('ALE/DonkeyKong-v5', render_mode='rgb_array')
obs. info = env.reset()

terminated = False
reward = 0

while(terminated == False and reward == 0):
    frames.append(env.render())  # 화면을 RGB 배열로 받아 리스트에 추가
    action = env.action_space.sample()  # 무작위 행동 선택
    observation, reward, terminated, truncated, info = env.step(action)

frames.append(env.render())
env.close()

# 동영상 파일로 저장
with imageio.get_writer('/content/drive/MyDrive/동영상/my_game.mp4', fps=30) as video:
    for frame in frames:
        video.append_data(frame)